<a href="https://colab.research.google.com/github/yongsa-nut/TU_CN409_GenAI_67_2/blob/main/CN_409_Claude_Tool_Use_Demo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Claude Tool Use Demo
- Materials are from https://github.com/anthropics/anthropic-cookbook and https://learn.deeplearning.ai/courses/building-toward-computer-use-with-anthropic/lesson/mshe8/tool-use

## Anthropic API

In [ ]:
!pip install anthropic

In [ ]:
import anthropic
from google.colab import userdata

SONNET = "claude-3-7-sonnet-20250219"
client = anthropic.Anthropic(
    api_key=userdata.get('anthropic'),
)

## Example 1: Using a Calculator Tool with Claude

### Setup a simple calculator tool
- We will simply create a function to do a simple arthmetic calculation.
- This function is for demonstration only. The use of `eval` is not recommended.

In [ ]:
import re

def calculate(expression):
    # Remove any non-digit or non-operator characters from the expression
    expression = re.sub(r'[^0-9+\-*/().]', '', expression)

    try:
        # Evaluate the expression using the built-in eval() function
        result = eval(expression)
        return str(result)
    except (SyntaxError, ZeroDivisionError, NameError, TypeError, OverflowError):
        return "Error: Invalid expression"

tools = [
    {
        "name": "calculator",
        "description": "A simple calculator that performs basic arithmetic operations.",
        "input_schema": {
            "type": "object",
            "properties": {
                "expression": {
                    "type": "string",
                    "description": "The mathematical expression to evaluate (e.g., '2 + 3 * 4')."
                }
            },
            "required": ["expression"]
        }
    }
]

In [ ]:
user_message = "What is the result of 1,000 * 9,343,116?"
response = client.messages.create(
        model=SONNET,
        max_tokens=4096,
        messages=[{"role": "user", "content": user_message}],
        tools=tools,
    )
print(response)

In [ ]:
tool_use = next(block for block in response.content if block.type == "tool_use")
tool_input = tool_use.input
tool_result = calculate(tool_input["expression"])
print(tool_result)

In [ ]:
response = client.messages.create(
            model=SONNET,
            max_tokens=4096,
            messages=[
                {"role": "user", "content": user_message},
                {"role": "assistant", "content": response.content},
                {
                    "role": "user",
                    "content": [
                        {
                            "type": "tool_result",
                            "tool_use_id": tool_use.id,
                            "content": tool_result,
                        }
                    ],
                },
            ],
            tools=tools,
        )
print(response)

In [ ]:
print(response.content[0].text)

### Interact with Claude

In [ ]:
# A simple function to call our calculator function/tool
def process_tool_call(tool_name, tool_input):
    if tool_name == "calculator":
        return calculate(tool_input["expression"])

# A function to call claude
def chat_with_claude(user_message, model_name=SONNET):
    print(f"\n{'='*50}\nUser Message: {user_message}\n{'='*50}")

    # Get the response from claude
    message = client.messages.create(
        model=model_name,
        max_tokens=4096,
        messages=[{"role": "user", "content": user_message}],
        tools=tools,
    )

    print(f"\nInitial Response:")
    print(f"Stop Reason: {message.stop_reason}")
    print(f"Content: {message.content}")

    if message.stop_reason == "tool_use":
        # get the first tool_use block
        tool_use = next(block for block in message.content if block.type == "tool_use")
        tool_name = tool_use.name
        tool_input = tool_use.input

        print(f"\nTool Used: {tool_name}")
        print(f"Tool Input: {tool_input}")

        # Get the result from the tool
        tool_result = process_tool_call(tool_name, tool_input)

        print(f"Tool Result: {tool_result}")

        # Create a new message with the tool's results
        response = client.messages.create(
            model=model_name,
            max_tokens=4096,
            messages=[
                {"role": "user", "content": user_message},
                {"role": "assistant", "content": message.content},
                {
                    "role": "user",
                    "content": [
                        {
                            "type": "tool_result",
                            "tool_use_id": tool_use.id,
                            "content": tool_result,
                        }
                    ],
                },
            ],
            tools=tools,
        )
    else:
        # no tool are used
        response = message

    print(f"\nFinal Response: {response.content[0].text}")


### Let's try it out

In [ ]:
chat_with_claude("What is the result of 1,984,135 * 9,343,116?")

In [ ]:
chat_with_claude("Calculate (12851 - 593) * 301 + 76")

In [ ]:
chat_with_claude("What is 15910385 divided by 193053?")

## Example 2: A Customer Service Agent
- We will create a mockup customer service agent that can look up customer information, retrieve order details, and cancel orders on behalf of the custome.

### Step1:  Simulate synthetic tool responses

In [ ]:
def get_customer_info(customer_id):
    # Simulated customer data
    customers = {
        "C1": {"name": "John Doe", "email": "john@example.com", "phone": "123-456-7890"},
        "C2": {"name": "Jane Smith", "email": "jane@example.com", "phone": "987-654-3210"}
    }
    return customers.get(customer_id, "Customer not found")

def get_order_details(order_id):
    # Simulated order data
    orders = {
        "O1": {"id": "O1", "product": "Widget A", "quantity": 2, "price": 19.99, "status": "Shipped"},
        "O2": {"id": "O2", "product": "Gadget B", "quantity": 1, "price": 49.99, "status": "Processing"}
    }
    return orders.get(order_id, "Order not found")

def cancel_order(order_id):
    # Simulated order cancellation
    if order_id in ["O1", "O2"]:
        return True
    else:
        return False

### Step 2: Define the client-side tools

In [ ]:
tools = [
    {
        "name": "get_customer_info",
        "description": "Retrieves customer information based on their customer ID. Returns the customer's name, email, and phone number.",
        "input_schema": {
            "type": "object",
            "properties": {
                "customer_id": {
                    "type": "string",
                    "description": "The unique identifier for the customer."
                }
            },
            "required": ["customer_id"]
        }
    },
    {
        "name": "get_order_details",
        "description": "Retrieves the details of a specific order based on the order ID. Returns the order ID, product name, quantity, price, and order status.",
        "input_schema": {
            "type": "object",
            "properties": {
                "order_id": {
                    "type": "string",
                    "description": "The unique identifier for the order."
                }
            },
            "required": ["order_id"]
        }
    },
    {
        "name": "cancel_order",
        "description": "Cancels an order based on the provided order ID. Returns a confirmation message if the cancellation is successful.",
        "input_schema": {
            "type": "object",
            "properties": {
                "order_id": {
                    "type": "string",
                    "description": "The unique identifier for the order to be cancelled."
                }
            },
            "required": ["order_id"]
        }
    }
]

### Step 3: Process tool calls and return results

In [ ]:
# a function to process the tool calls made by Claude and return the appropriate results
def process_tool_call(tool_name, tool_input):
    if tool_name == "get_customer_info":
        return get_customer_info(tool_input["customer_id"])
    elif tool_name == "get_order_details":
        return get_order_details(tool_input["order_id"])
    elif tool_name == "cancel_order":
        return cancel_order(tool_input["order_id"])

### Step 4: Interact with the chatbot

In [ ]:
import json

def chatbot_interaction(user_message):
    print(f"\n{'='*50}\nUser Message: {user_message}\n{'='*50}")

    messages = [
        {"role": "user", "content": user_message}
    ]
    # Get the initial response
    response = client.messages.create(
        model=SONNET,
        max_tokens=4096,
        tools=tools,
        messages=messages
    )

    print(f"\nInitial Response:")
    print(f"Stop Reason: {response.stop_reason}")
    print(f"Content: {response.content}")

    while response.stop_reason == "tool_use":
        tool_use = next(block for block in response.content if block.type == "tool_use")
        tool_name = tool_use.name
        tool_input = tool_use.input

        print(f"\nTool Used: {tool_name}")
        print(f"Tool Input:")
        print(json.dumps(tool_input, indent=2))

        tool_result = process_tool_call(tool_name, tool_input)

        print(f"\nTool Result:")
        print(json.dumps(tool_result, indent=2))

        messages = [
            {"role": "user", "content": user_message},
            {"role": "assistant", "content": response.content},
            {
                "role": "user",
                "content": [
                    {
                        "type": "tool_result",
                        "tool_use_id": tool_use.id,
                        "content": str(tool_result),
                    }
                ],
            },
        ]

        response = client.messages.create(
            model=SONNET,
            max_tokens=4096,
            tools=tools,
            messages=messages
        )

        print(f"\nResponse:")
        print(f"Stop Reason: {response.stop_reason}")
        print(f"Content: {response.content}")

    final_response = next(
        (block.text for block in response.content if hasattr(block, "text")),
        None,
    )

    print(f"\nFinal Response: {final_response}")


### Step 5: Test the chatbot

In [ ]:
chatbot_interaction("Can you tell me the email address for customer C1?")

In [ ]:
chatbot_interaction("What is the status of order O2?")

In [ ]:
chatbot_interaction("Please cancel order O1 for me.")

## Example 3: More Complete Customer Database


### Fake Database

In [ ]:
class FakeDatabase:
    def __init__(self):
        self.customers = [
            {"id": "1213210", "name": "John Doe", "email": "john@gmail.com", "phone": "123-456-7890", "username": "johndoe"},
            {"id": "2837622", "name": "Priya Patel", "email": "priya@candy.com", "phone": "987-654-3210", "username": "priya123"},
            {"id": "3924156", "name": "Liam Nguyen", "email": "lnguyen@yahoo.com", "phone": "555-123-4567", "username": "liamn"},
            {"id": "4782901", "name": "Aaliyah Davis", "email": "aaliyahd@hotmail.com", "phone": "111-222-3333", "username": "adavis"},
            {"id": "5190753", "name": "Hiroshi Nakamura", "email": "hiroshi@gmail.com", "phone": "444-555-6666", "username": "hiroshin"},
            {"id": "6824095", "name": "Fatima Ahmed", "email": "fatimaa@outlook.com", "phone": "777-888-9999", "username": "fatimaahmed"},
            {"id": "7135680", "name": "Alejandro Rodriguez", "email": "arodriguez@protonmail.com", "phone": "222-333-4444", "username": "alexr"},
            {"id": "8259147", "name": "Megan Anderson", "email": "megana@gmail.com", "phone": "666-777-8888", "username": "manderson"},
            {"id": "9603481", "name": "Kwame Osei", "email": "kwameo@yahoo.com", "phone": "999-000-1111", "username": "kwameo"},
            {"id": "1057426", "name": "Mei Lin", "email": "meilin@gmail.com", "phone": "333-444-5555", "username": "mlin"}
        ]

        self.orders = [
            {"id": "24601", "customer_id": "1213210", "product": "Wireless Headphones", "quantity": 1, "price": 79.99, "status": "Shipped"},
            {"id": "13579", "customer_id": "1213210", "product": "Smartphone Case", "quantity": 2, "price": 19.99, "status": "Processing"},
            {"id": "97531", "customer_id": "2837622", "product": "Bluetooth Speaker", "quantity": 1, "price": "49.99", "status": "Shipped"},
            {"id": "86420", "customer_id": "3924156", "product": "Fitness Tracker", "quantity": 1, "price": 129.99, "status": "Delivered"},
            {"id": "54321", "customer_id": "4782901", "product": "Laptop Sleeve", "quantity": 3, "price": 24.99, "status": "Shipped"},
            {"id": "19283", "customer_id": "5190753", "product": "Wireless Mouse", "quantity": 1, "price": 34.99, "status": "Processing"},
            {"id": "74651", "customer_id": "6824095", "product": "Gaming Keyboard", "quantity": 1, "price": 89.99, "status": "Delivered"},
            {"id": "30298", "customer_id": "7135680", "product": "Portable Charger", "quantity": 2, "price": 29.99, "status": "Shipped"},
            {"id": "47652", "customer_id": "8259147", "product": "Smartwatch", "quantity": 1, "price": 199.99, "status": "Processing"},
            {"id": "61984", "customer_id": "9603481", "product": "Noise-Cancelling Headphones", "quantity": 1, "price": 149.99, "status": "Shipped"},
            {"id": "58243", "customer_id": "1057426", "product": "Wireless Earbuds", "quantity": 2, "price": 99.99, "status": "Delivered"},
            {"id": "90357", "customer_id": "1213210", "product": "Smartphone Case", "quantity": 1, "price": 19.99, "status": "Shipped"},
            {"id": "28164", "customer_id": "2837622", "product": "Wireless Headphones", "quantity": 2, "price": 79.99, "status": "Processing"}
        ]

    def get_user(self, key, value):
        if key in {"email", "phone", "username"}:
            for customer in self.customers:
                if customer[key] == value:
                    return customer
            return f"Couldn't find a user with {key} of {value}"
        else:
            raise ValueError(f"Invalid key: {key}")

        return None

    def get_order_by_id(self, order_id):
        for order in self.orders:
            if order["id"] == order_id:
                return order
        return None

    def get_customer_orders(self, customer_id):
        return [order for order in self.orders if order["customer_id"] == customer_id]

    def cancel_order(self, order_id):
        order = self.get_order_by_id(order_id)
        if order:
            if order["status"] == "Processing":
                order["status"] = "Cancelled"
                return "Cancelled the order"
            else:
                return "Order has already shipped.  Can't cancel it."
        return "Can't find that order!"

In [ ]:
db = FakeDatabase()

In [ ]:
db.get_user("email","john@gmail.com")

{'id': '1213210',
 'name': 'John Doe',
 'email': 'john@gmail.com',
 'phone': '123-456-7890',
 'username': 'johndoe'}

In [ ]:
db.get_customer_orders('1213210')

In [ ]:
db.cancel_order('13579')

### Tools

In [ ]:
tools = [
    {
        "name": "get_user",
        "description": "Looks up a user by email, phone, or username.",
        "input_schema": {
            "type": "object",
            "properties": {
                "key": {
                    "type": "string",
                    "enum": ["email", "phone", "username"],
                    "description": "The attribute to search for a user by (email, phone, or username)."
                },
                "value": {
                    "type": "string",
                    "description": "The value to match for the specified attribute."
                }
            },
            "required": ["key", "value"]
        }
    },
    {
        "name": "get_order_by_id",
        "description": "Retrieves the details of a specific order based on the order ID. Returns the order ID, product name, quantity, price, and order status.",
        "input_schema": {
            "type": "object",
            "properties": {
                "order_id": {
                    "type": "string",
                    "description": "The unique identifier for the order."
                }
            },
            "required": ["order_id"]
        }
    },
    {
        "name": "get_customer_orders",
        "description": "Retrieves the list of orders belonging to a user based on a user's customer id.",
        "input_schema": {
            "type": "object",
            "properties": {
                "customer_id": {
                    "type": "string",
                    "description": "The customer_id belonging to the user"
                }
            },
            "required": ["customer_id"]
        }
    },
    {
        "name": "cancel_order",
        "description": "Cancels an order based on a provided order_id.  Only orders that are 'processing' can be cancelled",
        "input_schema": {
            "type": "object",
            "properties": {
                "order_id": {
                    "type": "string",
                    "description": "The order_id pertaining to a particular order"
                }
            },
            "required": ["order_id"]
        }
    }
]

### Giving Claude Tools

In [ ]:
messages = [
    {
        "role": "user",
        "content": "Show me my orders.  My username is priya123"
    }
]
response = client.messages.create(
    model=SONNET,
    max_tokens=4096,
    tools=tools,
    messages=messages
)

In [ ]:
print(response.content)

[TextBlock(citations=None, text="I'll help you look up your orders. Let me first get your customer details using your username, and then I'll retrieve your orders.", type='text'), ToolUseBlock(id='toolu_018pxCgGw2GbJoUwKoCcbpzp', input={'key': 'username', 'value': 'priya123'}, name='get_user', type='tool_use')]


In [ ]:
print(response.content[-1])

ToolUseBlock(id='toolu_018pxCgGw2GbJoUwKoCcbpzp', input={'key': 'username', 'value': 'priya123'}, name='get_user', type='tool_use')


In [ ]:
def process_tool_call(tool_name, tool_input):
    if tool_name == "get_user":
        return db.get_user(tool_input["key"], tool_input["value"])
    elif tool_name == "get_order_by_id":
        return db.get_order_by_id(tool_input["order_id"])
    elif tool_name == "get_customer_orders":
        return db.get_customer_orders(tool_input["customer_id"])
    elif tool_name == "cancel_order":
        return db.cancel_order(tool_input["order_id"])

In [ ]:
tool_use = response.content[-1]
tool_name = tool_use.name
tool_input = tool_use.input

In [ ]:
tool_result = process_tool_call(tool_name, tool_input)

In [ ]:
{
    "role": "user",
    "content": [
        {
            "type": "tool_result",
            "tool_use_id": tool_use.id,
            "content": str(tool_result),
        }
    ],
}

{'role': 'user',
 'content': [{'type': 'tool_result',
   'tool_use_id': 'toolu_018pxCgGw2GbJoUwKoCcbpzp',
   'content': "{'id': '2837622', 'name': 'Priya Patel', 'email': 'priya@candy.com', 'phone': '987-654-3210', 'username': 'priya123'}"}]}

In [ ]:
def extract_reply(text):
    pattern = r'<reply>(.*?)</reply>'
    match = re.search(pattern, text, re.DOTALL)
    if match:
        return match.group(1)
    else:
        return None

def simple_chat():
    system_prompt = """
    You are a customer support chat bot for an online retailer
    called Acme Co.Your job is to help users look up their account,
    orders, and cancel orders.Be helpful and brief in your responses.
    You have access to a set of tools, but only use them when needed.
    If you do not have enough information to use a tool correctly,
    ask a user follow up questions to get the required inputs.
    Do not call any of the tools unless you have the required
    data from a user.

    In each conversational turn, you will begin by thinking about
    your response. Once you're done, you will write a user-facing
    response. It's important to place all user-facing conversational
    responses in <reply></reply> XML tags to make them easy to parse.
    """
    user_message = input("\nUser: ")
    messages = [{"role": "user", "content": user_message}]
    while True:
        if user_message == "quit":
            break
        #If the last message is from the assistant,
        # get another input from the user
        if messages[-1].get("role") == "assistant":
            user_message = input("\nUser: ")
            messages.append({"role": "user", "content": user_message})

        #Send a request to Claude
        response = client.messages.create(
            model=SONNET,
            system=system_prompt,
            max_tokens=4096,
            tools=tools,
            messages=messages
        )
        # Update messages to include Claude's response
        messages.append(
            {"role": "assistant", "content": response.content}
        )

        #If Claude stops because it wants to use a tool:
        if response.stop_reason == "tool_use":
            #Naive approach assumes only 1 tool is called at a time
            tool_use = response.content[-1]
            tool_name = tool_use.name
            tool_input = tool_use.input
            print(f"=====Claude wants to use the {tool_name} tool=====")


            #Actually run the underlying tool functionality on our db
            tool_result = process_tool_call(tool_name, tool_input)

            #Add our tool_result message:
            messages.append(
                {
                    "role": "user",
                    "content": [
                        {
                            "type": "tool_result",
                            "tool_use_id": tool_use.id,
                            "content": str(tool_result),
                        }
                    ],
                },
            )
        else:
            #If Claude does NOT want to use a tool,
            #just print out the text reponse
            model_reply = extract_reply(response.content[0].text)
            print("\nAcme Co Support: " + f"{model_reply}" )

In [ ]:
simple_chat()


User: Hello

Acme Co Support: Hello! I'm the Acme Co. customer service assistant. I can help you look up your account information, check your orders, or help with order cancellations. How can I assist you today?

User: could you check my order? my username is johndoe
=====Claude wants to use the get_user tool=====
=====Claude wants to use the get_customer_orders tool=====

Acme Co Support: I found your orders, John! You have 3 orders:
1. Wireless Headphones (Order #24601) - Shipped
2. 2x Smartphone Cases (Order #13579) - Processing
3. Smartphone Case (Order #90357) - Shipped

Would you like more details about any specific order?

User: Can you cancel my second order?
=====Claude wants to use the cancel_order tool=====

Acme Co Support: I've successfully cancelled your order #13579 for 2 Smartphone Cases. Is there anything else I can help you with?

User: Check my order again
=====Claude wants to use the get_customer_orders tool=====

Acme Co Support: Here are your current orders:
1. W

## Extracting Structured JSON using Claude and Tool Use
- You can use tool use to extracting structured JSON (instead of prompting it directly)

### Example 1: Article Summarization
- In this example, we'll use Claude to generate a JSON summary of an article, including fields for the author, topics, summary, coherence score, persuasion score, and a counterpoint.

In [ ]:
import requests
from bs4 import BeautifulSoup
import json
tools = [
    {
        "name": "print_summary",
        "description": "Prints a summary of the article.",
        "input_schema": {
            "type": "object",
            "properties": {
                "author": {"type": "string", "description": "Name of the article author"},
                "topics": {
                    "type": "array",
                    "items": {"type": "string"},
                    "description": 'Array of topics, e.g. ["tech", "politics"]. Should be as specific as possible, and can overlap.'
                },
                "summary": {"type": "string", "description": "Summary of the article. One or two paragraphs max."},
                "coherence": {"type": "integer", "description": "Coherence of the article's key points, 0-100 (inclusive)"},
                "persuasion": {"type": "number", "description": "Article's persuasion score, 0.0-1.0 (inclusive)"}
            },
            "required": ['author', 'topics', 'summary', 'coherence', 'persuasion', 'counterpoint']
        }
    }
]

url = "https://www.anthropic.com/news/third-party-testing"
response = requests.get(url)
soup = BeautifulSoup(response.text, "html.parser")
article = " ".join([p.text for p in soup.find_all("p")])

query = f"""
<article>
{article}
</article>

Use the `print_summary` tool.
"""

response = client.messages.create(
    model=SONNET,
    max_tokens=4096,
    tools=tools,
    messages=[{"role": "user", "content": query}]
)
json_summary = None
for content in response.content:
    if content.type == "tool_use" and content.name == "print_summary":
        json_summary = content.input
        break

if json_summary:
    print("JSON Summary:")
    print(json.dumps(json_summary, indent=2))
else:
    print("No JSON summary found in the response.")

### Example 2: Named Entity Recognition
- In this example, we'll use Claude to perform named entity recognition on a given text and return the entities in a structured JSON format.



In [ ]:
tools = [
    {
        "name": "print_entities",
        "description": "Prints extract named entities.",
        "input_schema": {
            "type": "object",
            "properties": {
                "entities": {
                    "type": "array",
                    "items": {
                        "type": "object",
                        "properties": {
                            "name": {"type": "string", "description": "The extracted entity name."},
                            "type": {"type": "string", "description": "The entity type (e.g., PERSON, ORGANIZATION, LOCATION)."},
                            "context": {"type": "string", "description": "The context in which the entity appears in the text."}
                        },
                        "required": ["name", "type", "context"]
                    }
                }
            },
            "required": ["entities"]
        }
    }
]

text = "John works at Google in New York. He met with Sarah, the CEO of Acme Inc., last week in San Francisco."

query = f"""
<document>
{text}
</document>

Use the print_entities tool.
"""

response = client.messages.create(
    model=SONNET,
    max_tokens=4096,
    tools=tools,
    messages=[{"role": "user", "content": query}]
)

json_entities = None
for content in response.content:
    if content.type == "tool_use" and content.name == "print_entities":
        json_entities = content.input
        break

if json_entities:
    print("Extracted Entities (JSON):")
    print(json.dumps(json_entities, indent=2))
else:
    print("No entities found in the response.")

### Example 3: Sentiment Analysis
- In this example, we'll use Claude to perform sentiment analysis on a given text and return the sentiment scores in a structured JSON format.



In [ ]:
tools = [
    {
        "name": "print_sentiment_scores",
        "description": "Prints the sentiment scores of a given text.",
        "input_schema": {
            "type": "object",
            "properties": {
                "positive_score": {"type": "number", "description": "The positive sentiment score, ranging from 0.0 to 1.0."},
                "negative_score": {"type": "number", "description": "The negative sentiment score, ranging from 0.0 to 1.0."},
                "neutral_score": {"type": "number", "description": "The neutral sentiment score, ranging from 0.0 to 1.0."}
            },
            "required": ["positive_score", "negative_score", "neutral_score"]
        }
    }
]

text = "The product was okay, but the customer service was terrible. I probably won't buy from them again."

query = f"""
<text>
{text}
</text>

Use the print_sentiment_scores tool.
"""

response = client.messages.create(
    model=SONNET,
    max_tokens=4096,
    tools=tools,
    messages=[{"role": "user", "content": query}]
)

json_sentiment = None
for content in response.content:
    if content.type == "tool_use" and content.name == "print_sentiment_scores":
        json_sentiment = content.input
        break

if json_sentiment:
    print("Sentiment Analysis (JSON):")
    print(json.dumps(json_sentiment, indent=2))
else:
    print("No sentiment analysis found in the response.")

### Example 4: Working with unknown keys
- In some cases you may not know the exact JSON object shape up front. In this example we provide an open ended input_schema and instruct Claude via prompting how to interact with the tool.


In [ ]:
tools = [
    {
        "name": "print_all_characteristics",
        "description": "Prints all characteristics which are provided.",
        "input_schema": {
            "type": "object",
            "additionalProperties": True
        }
    }
]

query = f"""Given a description of a character, your task is to extract all the characteristics of the character and print them using the print_all_characteristics tool.

The print_all_characteristics tool takes an arbitrary number of inputs where the key is the characteristic name and the value is the characteristic value (age: 28 or eye_color: green).

<description>
The man is tall, with a beard and a scar on his left cheek. He has a deep voice and wears a black leather jacket.
</description>

Now use the print_all_characteristics tool."""

response = client.messages.create(
    model=SONNET,
    max_tokens=4096,
    tools=tools,
    tool_choice={"type": "tool", "name": "print_all_characteristics"},
    messages=[{"role": "user", "content": query}]
)

tool_output = None
for content in response.content:
    if content.type == "tool_use" and content.name == "print_all_characteristics":
        tool_output = content.input
        break

if tool_output:
    print("Characteristics (JSON):")
    print(json.dumps(tool_output, indent=2))
else:
    print("Something went wrong.")

### Example 5: Extract JSON from images

In [ ]:
nutrition_tool = {
    "name": "print_nutrition_info",
    "description": "Extracts nutrition information from an image of a nutrition label",
    "input_schema": {
        "type": "object",
        "properties": {
            "calories": {"type": "integer", "description": "The number of calories per serving"},
            "total_fat": {"type": "integer", "description": "The amount of total fat in grams per serving"},
            "cholesterol": {"type": "integer", "description": "The amount of cholesterol in milligrams per serving"},
            "total_carbs": {"type": "integer", "description": "The amount of total carbohydrates in grams per serving"},
            "protein": {"type": "integer", "description": "The amount of protein in grams per serving"}
        },
        "required": ["calories", "total_fat", "cholesterol", "total_carbs", "protein"]
    }
}

In [ ]:
!wget https://raw.githubusercontent.com/anthropics/anthropic-cookbook/main/images/tool_use/nutrition_label.png

In [ ]:
from IPython.display import Image
import base64

Image(filename='/content/nutrition_label.png')

In [ ]:
def get_base64_encoded_image(image_path):
    with open(image_path, "rb") as image_file:
        binary_data = image_file.read()
        base_64_encoded_data = base64.b64encode(binary_data)
        base64_string = base_64_encoded_data.decode('utf-8')
        return base64_string

message_list = [
    {
        "role": "user",
        "content": [
            {"type": "image",
             "source": {"type": "base64", "media_type": "image/png",
                        "data": get_base64_encoded_image("/content/nutrition_label.png")}},
            {"type": "text",
             "text": "Please print the nutrition information from this nutrition label image."}
        ]
    }
]

response = client.messages.create(
    model=SONNET,
    max_tokens=4096,
    messages=message_list,
    tools=[nutrition_tool]
)

if response.stop_reason == "tool_use":
    last_content_block = response.content[-1]
    if last_content_block.type == 'tool_use':
        tool_name = last_content_block.name
        tool_inputs = last_content_block.input
        print(f"=======Claude Wants To Call The {tool_name} Tool=======")
        print(tool_inputs)

else:
    print("No tool was called. This shouldn't happen!")

## Tool Choice

When working with the `tool_choice` parameter, we have three possible options:

- `auto` allows Claude to decide whether to call any provided tools or not
- `tool` allows us to force Claude to always use a particular tool
- `any` tells Claude that it must use one of the provided tools, but doesn't force a particular tool
Let's take a look at each option in detail. We'll start by importing the Anthropic SDK:

### Auto
- Setting tool_choice to `auto` allows the model to automatically decide whether to use tools or not. This is the default behavior when working with tools.

- Let's start by defining a tool called `web_search`. Please note, to keep this demo simple, we're not actually searching the web here:

In [ ]:
def web_search(topic):
    print(f"pretending to search the web for {topic}")

web_search_tool = {
    "name": "web_search",
    "description": "A tool to retrieve up to date information on a given topic by searching the web",
    "input_schema": {
        "type": "object",
        "properties": {
            "topic": {
                "type": "string",
                "description": "The topic to search the web for"
            },
        },
        "required": ["topic"]
    }
}

Next, we write a function that accepts a `user_query` and passes it along to Claude, along with the `web_search_tool`.

We also set `tool_choice` to `auto`:

```python
tool_choice={"type": "auto"}
```

Here's the complete function:

In [ ]:
from datetime import date

def chat_with_web_search(user_query):
    messages = [{"role": "user", "content": user_query}]

    system_prompt=f"""
    Answer as many questions as you can using your existing knowledge.
    Only search the web for queries that you can not confidently answer.
    Today's date is {date.today().strftime("%B %d %Y")}
    If you think a user's question involves something in the future that hasn't happened yet, use the search tool.
    """

    response = client.messages.create(
        system=system_prompt,
        model=SONNET,
        messages=messages,
        max_tokens=1000,
        tool_choice={"type": "auto"},
        tools=[web_search_tool]
    )
    last_content_block = response.content[-1]
    if last_content_block.type == "text":
        print("Claude did NOT call a tool")
        print(f"Assistant: {last_content_block.text}")
    elif last_content_block.type == "tool_use":
        print("Claude wants to use a tool")
        print(last_content_block)

- Let's test it out.

In [ ]:
chat_with_web_search("What color is the sky?")

In [ ]:
chat_with_web_search("Who won the 2024 Miami Grand Prix?")

In [ ]:
# Claude should NOT need to use the tool for this:
chat_with_web_search("Who won the superbowl in 2022?")

In [ ]:
# Claude SHOULD use the tool for this:
chat_with_web_search("Who won the superbowl in 2024?")

### Tool
- We can force Claude to use a particular tool using `tool_choice`. In the example below, we've defined two simple tools:

In [ ]:
tools = [
    {
        "name": "print_sentiment_scores",
        "description": "Prints the sentiment scores of a given tweet or piece of text.",
        "input_schema": {
            "type": "object",
            "properties": {
                "positive_score": {"type": "number", "description": "The positive sentiment score, ranging from 0.0 to 1.0."},
                "negative_score": {"type": "number", "description": "The negative sentiment score, ranging from 0.0 to 1.0."},
                "neutral_score": {"type": "number", "description": "The neutral sentiment score, ranging from 0.0 to 1.0."}
            },
            "required": ["positive_score", "negative_score", "neutral_score"]
        }
    },
    {
        "name": "calculator",
        "description": "Adds two number",
        "input_schema": {
            "type": "object",
            "properties": {
                "num1": {"type": "number", "description": "first number to add"},
                "num2": {"type": "number", "description": "second number to add"},
            },
            "required": ["num1", "num2"]
        }
    }
]

#### Testing `auto`
In this first "bad" version of the `analyze_tweet_sentiment` function, we provide Claude with both tools. For the sake of comparison, we'll start by setting tool_choice to "auto":

```python
tool_choice={"type": "auto"}
```

Please note that we are deliberately not providing Claude with a well-written prompt, to make it easier to see the impact of forcing the use of a particular tool.

In [ ]:
def analyze_tweet_sentiment(query):
    response = client.messages.create(
        model=SONNET,
        max_tokens=4096,
        tools=tools,
        tool_choice={"type": "auto"},
        messages=[{"role": "user", "content": query}]
    )
    print(response)

Let's see what happens when we call the function with the tweet "Holy cow, I just made the most incredible meal!"

In [ ]:
analyze_tweet_sentiment("Holy cow, I just made the most incredible meal!")

Message(id='msg_vrtx_01VaAMu5uRpAYNXqFE7kw38L', content=[TextBlock(text="That's wonderful! I'm so glad to hear you had such a great culinary experience. It sounds like you're feeling quite excited and positive about your meal. \n\nWhile I don't have specific details about your meal, I can certainly analyze the sentiment of your statement. Would you like me to use a sentiment analysis tool to quantify the positivity in your exclamation? This could give us a more precise measure of the enthusiasm in your words.", type='text')], model='claude-3-5-sonnet-20240620', role='assistant', stop_reason='end_turn', stop_sequence=None, type='message', usage=Usage(input_tokens=564, output_tokens=96))


Next, let's imagine someone tweets this: "I love my cats! I had four and just adopted 2 more! Guess how many I have now?"

In [ ]:
analyze_tweet_sentiment("I love my cats! I had four and just adopted 2 more! Guess how many I have now?")

#### Testing `tool`

Next, let's force Claude to always use the `print_sentiment_scores` tool by updating `tool_choice`:
```python
tool_choice={"type": "tool", "name": "print_sentiment_scores"}
```
In addition to setting `type` to `tool`, we must provide a particular tool name.

In [ ]:
def analyze_tweet_sentiment(query):
    response = client.messages.create(
        model=SONNET,
        max_tokens=4096,
        tools=tools,
        tool_choice={"type": "tool", "name": "print_sentiment_scores"},
        messages=[{"role": "user", "content": query}]
    )
    print(response) #response.content[0].input

Now let's repeat the experiment again.

In [ ]:
analyze_tweet_sentiment("Holy cow, I just made the most incredible meal!")

{'positive_score': 0.9, 'negative_score': 0.0, 'neutral_score': 0.1}


In [ ]:
analyze_tweet_sentiment("I love my cats! I had four and just adopted 2 more! Guess how many I have now?")

{'positive_score': 0.8, 'negative_score': 0.0, 'neutral_score': 0.2}


Even though we're forcing Claude to call our `print_sentiment_scores` tool, we should still employ some basic prompt engineering:

In [ ]:
def analyze_tweet_sentiment(query):

    prompt = f"""
    Analyze the sentiment in the following tweet:
    <tweet>{query}</tweet>
    """

    response = client.messages.create(
        model=SONNET,
        max_tokens=4096,
        tools=tools,
        tool_choice={"type": "auto"},
        messages=[{"role": "user", "content": prompt}]
    )
    print(response)

### Any

The final option for `tool_choice` is `any` which allows us to tell Claude "you must call a tool, but you can pick which one". Imagine we want to create a SMS chatbot using Claude. The only way for this chatbot to actually "communicate" with a user is via SMS text message.

In the example below, we make a very simple text-messaging assistant that has access to two tools:

- `send_text_to_user` sends a text message to a user
- `get_customer_info` looks up customer data based on a username

The idea is to create a chatbot that always calls one of these tools and never responds with a non-tool response. In all situations, Claude should either respond back by trying to send a text message or calling `get_customer_info` to get more customer information.

Most importantly, we set `tool_choice` to "any":

```python
tool_choice={"type": "any"}
```

In [ ]:
def send_text_to_user(text):
    # Sends a text to the user
    # We'll just print out the text to keep things simple:
    print(f"TEXT MESSAGE SENT: {text}")

def get_customer_info(username):
    return {
        "username": username,
        "email": f"{username}@email.com",
        "purchases": [
            {"id": 1, "product": "computer mouse"},
            {"id": 2, "product": "screen protector"},
            {"id": 3, "product": "usb charging cable"},
        ]
    }

tools = [
    {
        "name": "send_text_to_user",
        "description": "Sends a text message to a user",
        "input_schema": {
            "type": "object",
            "properties": {
                "text": {"type": "string", "description": "The piece of text to be sent to the user via text message"},
            },
            "required": ["text"]
        }
    },
    {
        "name": "get_customer_info",
        "description": ("gets information on a customer based on the customer's username. "+
                        "Response includes email, username, and previous purchases. "+
                        "Only call this tool once a user has provided you with their username"),
        "input_schema": {
            "type": "object",
            "properties": {
                "username": {"type": "string", "description": "The username of the user in question. "},
            },
            "required": ["username"]
        }
    },
]

system_prompt = """
All your communication with a user is done via text message.
Only call tools when you have enough information to accurately call them.
Do not call the get_customer_info tool until a user has provided you with their username. This is important.
If you do not know a user's username, simply ask a user for their username.
"""

def sms_chatbot(user_message):
    messages = [{"role": "user", "content":user_message}]

    response = client.messages.create(
        system=system_prompt,
        model=SONNET,
        max_tokens=4096,
        tools=tools,
        tool_choice={"type": "any"},
        messages=messages
    )
    if response.stop_reason == "tool_use":
        last_content_block = response.content[-1]
        if last_content_block.type == 'tool_use':
            tool_name = last_content_block.name
            tool_inputs = last_content_block.input
            print(f"=======Claude Wants To Call The {tool_name} Tool=======")
            if tool_name == "send_text_to_user":
                send_text_to_user(tool_inputs["text"])
            elif tool_name == "get_customer_info":
                print(get_customer_info(tool_inputs["username"]))
            else:
                print("Oh dear, that tool doesn't exist!")

    else:
        print("No tool was called. This shouldn't happen!")

#### Testing Any
- Let's start simple:

In [ ]:
sms_chatbot("Hey there! How are you?")

- Next, we'll ask Claude something a bit trickier:

In [ ]:
sms_chatbot("I need help looking up an order")

- Now, let's see what happens when we provide Claude with our username:



In [ ]:
sms_chatbot("I need help looking up an order.  My username is jenny76")

- Even if we send Claude a gibberish message, it will still call one of our tools:

In [ ]:
sms_chatbot("askdj aksjdh asjkdbhas kjdhas 1+1 ajsdh")